In [1]:
%bazel "@PythonKit//:PythonKit"
%bazel "@SwiftCrypto//:Crypto"
%bazel "@swift-jupyter//:JupyterDisplay"
%bazel "//mujoco:jupyter"

Installing packages:
|-@PythonKit//:PythonKit
|-@SwiftCrypto//:Crypto
|-@swift-jupyter//:JupyterDisplay
|-//mujoco:jupyter
Working in: /home/liu/Dropbox/workspace/sim2real/.ibzlnb/tWrsT
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-opt/bin/mujoco/MuJoCoJupyterExtensions.swiftmodule
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-opt/bin/external/PythonKit/PythonKit.swiftmodule
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-opt/bin/external/SwiftCrypto/Crypto.swiftmodule
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-opt/bin/external/SwiftNIO/NIO.swiftmodule
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-opt/bin/external/SwiftNIO/NIOConcurrencyHelpers.swiftmodule
swiftmodu

In [2]:
import MuJoCo
import MuJoCoJupyterExtensions

In [3]:
let simulate = Simulate(width: 960, height: 540, title: "simulate")
var m = try MjModel(fromXMLPath: "/home/liu/workspace/s4nnc/gym/assets/ant.xml")
var d = m.makeData()
simulate.use(model: m, data: &d)

In [4]:
let httpRenderServer = HTTPRenderServer(simulate, maxWidth: 960, maxHeight: 540, canResize: false)
let channel = try httpRenderServer.bind(host: "0.0.0.0", port: 12345).wait()

In [5]:
simulate.makeContext(hidden: true)

In [ ]:
JupyterDisplay.display(html: httpRenderServer.html)
JupyterDisplay.flush()

let syncmisalign: Double = 0.1  // maximum time mis-alignment before re-sync
let refreshfactor: Double = 0.7  // fraction of refresh available for simulation
var ctrlnoise: [Double]? = nil
let vmode = GLContext.videoMode
var simsync: Double = 0
var cpusync: Double = 0
while !simulate.exitrequest {
    simulate.yield()
    if simulate.running {
        let tmstart = GLContext.time
        if simulate.ctrlnoisestd > 0 {
            let rate = exp(-m.opt.timestep / simulate.ctrlnoiserate)
            let scale = simulate.ctrlnoisestd * (1 - rate * rate).squareRoot()
            let prevctrlnoise = ctrlnoise
            if ctrlnoise == nil {
                // allocate ctrlnoise
                ctrlnoise = Array(repeating: 0, count: Int(m.nu))
            }
            for i in 0..<Int(m.nu) {
                ctrlnoise?[i] = rate * (prevctrlnoise?[i] ?? 0) + scale * Simulate.normal(1)
                d.ctrl[i] = ctrlnoise?[i] ?? 0
            }
        }
        let offset = abs((d.time * Double(simulate.slowdown) - simsync) - (tmstart - cpusync))
        // Out of sync.
        if d.time * Double(simulate.slowdown) < simsync || tmstart < cpusync
            || cpusync == 0 || offset > syncmisalign * Double(simulate.slowdown)
            || simulate.speedchanged
        {
            cpusync = tmstart
            simsync = d.time * Double(simulate.slowdown)
            simulate.speedchanged = false
            d.xfrcApplied.zero()
            simulate.perturb.applyPerturbPose(model: m, data: &d, flgPaused: 0)
            simulate.perturb.applyPerturbForce(model: m, data: &d)
            m.step(data: &d)
        } else {
            // In sync.
            // step while simtime lags behind cputime, and within safefactor
            while d.time * Double(simulate.slowdown) - simsync < GLContext.time - cpusync
              && GLContext.time - tmstart < refreshfactor / Double(vmode.refreshRate)
            {
                // clear old perturbations, apply new
                d.xfrcApplied.zero()
                simulate.perturb.applyPerturbPose(model: m, data: &d, flgPaused: 0)
                simulate.perturb.applyPerturbForce(model: m, data: &d)
                let prevtm = d.time * Double(simulate.slowdown)
                m.step(data: &d)

                // break on reset
                if d.time * Double(simulate.slowdown) < prevtm {
                    break
                }
            }
        }
    } else {
        simulate.perturb.applyPerturbPose(model: m, data: &d, flgPaused: 1)
        m.forward(data: &d)
    }
}